In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Dense, GlobalAveragePooling2D, Lambda
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from glob import glob

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class CASIAGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size=(112, 112)):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.indices = np.arange(len(self.image_paths))

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        X = np.empty((self.batch_size, self.image_size[0], self.image_size[1], 3))
        y = np.empty((self.batch_size), dtype=int)

        for i, idx in enumerate(batch_indices):
            img = cv2.imread(self.image_paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #konversi ke rgb agar merata

            img = cv2.resize(img, self.image_size)
            img = (img.astype('float32') - 127.5) / 128.0

            X[i,] = img
            y[i] = self.labels[idx]

        return X, tf.keras.utils.to_categorical(y, num_classes=self.num_classes)

    def on_epoch_end(self):
        np.random.shuffle(self.indices)


def load_casia_data(base_dir="/content/drive/MyDrive/casia dataset"):
    all_image_paths = []
    all_labels_str = []


    id_folders = sorted(glob(os.path.join(base_dir, '*')))

    for folder in id_folders:
        person_id = os.path.basename(folder)

        image_files = glob(os.path.join(folder, '*.jpg')) + glob(os.path.join(folder, '*.png'))

        all_image_paths.extend(image_files)
        all_labels_str.extend([person_id] * len(image_files))


    le = LabelEncoder()
    all_labels_int = le.fit_transform(all_labels_str)

    train_paths, val_paths, train_labels, val_labels = train_test_split(
        all_image_paths, all_labels_int, test_size=0.1, stratify=all_labels_int
    )

    num_classes = len(le.classes_)
    return train_paths, train_labels, val_paths, val_labels, num_classes

In [4]:
class ArcFace(tf.keras.layers.Layer):
    def __init__(self, num_classes, s=64.0, m=0.5, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.s = s
        self.m = m
        self.regularizer = tf.keras.regularizers.get(regularizer)
        self.w = None

    def build(self, input_shape):
        # input_shape adalah list/tuple: [embedding_shape, label_shape]
        if isinstance(input_shape, (list, tuple)) and len(input_shape) > 1:
            embedding_shape = input_shape[0]
        else:
            embedding_shape = input_shape

        embedding_dim = embedding_shape[-1]

        self.w = self.add_weight(
            name='ArcFace_W',
            shape=(embedding_dim, self.num_classes),
            initializer='glorot_uniform',
            trainable=True,
            regularizer=self.regularizer
        )
        super(ArcFace, self).build(input_shape)

    def call(self, inputs):
        if isinstance(inputs, (list, tuple)):
            embedding_input = inputs[0]
            labels = inputs[1]
        else:
            embedding_input = inputs
            labels = None

        w = tf.nn.l2_normalize(self.w, axis=0)

        cosine = tf.matmul(embedding_input, w)

        if labels is None:
            return cosine * self.s

        theta = tf.acos(tf.clip_by_value(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
        marginal_target = tf.cos(theta + self.m)
        original_target = tf.cos(theta)

        output = tf.where(tf.cast(labels, dtype=tf.bool), marginal_target, original_target)
        output *= self.s

        return output

In [5]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    pass

def build_face_recognition_model(input_shape, num_classes, embedding_dim=512):
    inputs = Input(shape=input_shape)

    #Backbone(ResNet-50)
    backbone = tf.keras.applications.ResNet50(
        include_top=False,
        weights=None, #scratch pake None tapi kalau mau transfer learning pake model lain
        input_tensor=inputs,
        pooling=None
    )
    x = backbone.output

    x = GlobalAveragePooling2D()(x)

    #Feature Embedding Layer (L2 Normalized)
    x = Dense(embedding_dim, use_bias=False, kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)

    embedding = Lambda(lambda x: tf.nn.l2_normalize(x, axis=1), name='face_embedding')(x)

    label_input = Input(shape=(num_classes,), name='label_input')

    arcface_output = ArcFace(num_classes=num_classes, s=64.0, m=0.5)([embedding, label_input])

    model_training = Model(inputs=[inputs, label_input], outputs=arcface_output, name='ArcFace_Training_Model')

    model_verification = Model(inputs=inputs, outputs=embedding, name='Face_Verification_Model')

    return model_training, model_verification

In [6]:
IMAGE_SIZE = (112, 112)
BATCH_SIZE = 32
EMBEDDING_DIM = 512
BASE_DIR = "/content/drive/MyDrive/casia dataset"

train_paths, train_labels, val_paths, val_labels, NUM_CLASSES = load_casia_data(BASE_DIR)
print(f"Total Kelas (ID): {NUM_CLASSES}")

train_generator = CASIAGenerator(train_paths, train_labels, BATCH_SIZE, IMAGE_SIZE)
val_generator = CASIAGenerator(val_paths, val_labels, BATCH_SIZE, IMAGE_SIZE)

train_generator.num_classes = NUM_CLASSES
val_generator.num_classes = NUM_CLASSES

#model building
model_training, model_verification = build_face_recognition_model(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    num_classes=NUM_CLASSES,
    embedding_dim=EMBEDDING_DIM
)

#model compile
model_training.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

def combined_generator(data_generator):
    data_iterator = iter(data_generator)

    while True:
        try:
            X_batch, y_batch = next(data_iterator)
        except StopIteration:
            data_iterator = iter(data_generator)
            X_batch, y_batch = next(data_iterator)
        yield ((X_batch, y_batch), y_batch)
print("\nModel siap dilatih. Setelah pelatihan, gunakan 'model_verification' untuk aplikasi absensi.")
model_verification.summary()

Total Kelas (ID): 200

Model siap dilatih. Setelah pelatihan, gunakan 'model_verification' untuk aplikasi absensi.


Model: "Face_Verification_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 118, 118,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 56, 56,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 56, 56,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 56, 56,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 58, 58,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 28, 28,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 28, 28,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 28, 28,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 28, 28,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 28, 28,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 28, 28,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 28, 28,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 28, 28,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 28, 28,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 28, 28,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 28, 28,    │      1,024 │ conv2_block1_3_c

 Total params: 24,638,336 (93.99 MB)

 Trainable params: 24,584,192 (93.78 MB)

 Non-trainable params: 54,144 (211.50 KB)

In [8]:
# 6. Mulai Pelatihan
model_training.fit(
    combined_generator(train_generator),
    steps_per_epoch=len(train_generator),
    epochs=5, # Sesuaikan jumlah epoch
    validation_data=combined_generator(val_generator),
    validation_steps=len(val_generator)
)

Epoch 1/5
568/568 ━━━━━━━━━━━━━━━━━━━━ 6631s 12s/step - accuracy: 0.0000e+00 - loss: 15.3383 - val_accuracy: 0.0000e+00 - val_loss: 16.1708
Epoch 2/5
568/568 ━━━━━━━━━━━━━━━━━━━━ 4012s 7s/step - accuracy: 0.0000e+00 - loss: 16.1446 - val_accuracy: 0.0000e+00 - val_loss: 16.1206
Epoch 3/5
568/568 ━━━━━━━━━━━━━━━━━━━━ 3995s 7s/step - accuracy: 0.0000e+00 - loss: 16.1197 - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/5
568/568 ━━━━━━━━━━━━━━━━━━━━ 4007s 7s/step - accuracy: 0.0052 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/5
568/568 ━━━━━━━━━━━━━━━━━━━━ 4041s 7s/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan


In [9]:
model_verification.save('/content/drive/MyDrive/face_recognition_model.h5')
print("Model 'face_recognition_model.h5' saved to Google Drive.")

Model 'face_recognition_model.h5' saved to Google Drive.


In [10]:
model_verification.save('/content/drive/MyDrive/face_recognition_model.keras')